In [1]:
#@title Import e funzioni utili
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from numba import njit
from scipy.optimize import curve_fit
import scipy.stats

In [3]:
#@title Clean data
#create a list of all the files in the folder
DIR = "../data/energia/LOB_ottobre21/LOB_ottobre21/"
lob_files =os.listdir(DIR)
lob_files.sort()
lst_order = []
for element in lob_files:
    # import data
    df = pd.read_csv(DIR+ element)
    df.fillna(0, inplace = True)

    # delete first two column and empty LOB
    df.drop(columns = ["Unnamed: 0", "key"], inplace = True)
    df.drop(df[df["AskPrice_0"] == 0].index.to_list(), inplace = True)
    df.drop(df[df["BidPrice_0"] == 0].index.to_list(), inplace = True)

    # scale price to dollar cent and add mid price and spread
    df["BidPrice_0"] = df["BidPrice_0"]
    df["AskPrice_0"] = df["AskPrice_0"]
    df["MidPrice"] = (df["BidPrice_0"] + df["AskPrice_0"]) / 2
    df["Spread"] = df["AskPrice_0"] - df["BidPrice_0"]

    # transform the column Datetime from string to datetime
    df["Datetime"]= pd.to_datetime(df["Datetime"])

    #create a new column that represent second to midnight 
    seconds = np.zeros(len(df))
    for i, date in enumerate(df["Datetime"]):
        seconds[i] = date.second + 60 * date.minute + 3600 * date.hour + \
                                    date.microsecond * 1e-6
    df["Time"] = seconds

    #delete rows with spread <= 0
    df = df.loc[df["Spread"] > 0]
    
    df = df.loc[df["Datetime"].dt.hour > 6]
    df = df.loc[df["Datetime"].dt.hour < 16]
    
    df = df.loc[df["Datetime"].dt.day != 27]
    
    lst_order.append(df)

clean_data = pd.concat(lst_order)
clean_data.reset_index(inplace = True, drop = True)

df = clean_data.iloc[:,1:41].diff().fillna(0)

df["Price"] = df["AskVolume_0"]*0
df["Volume"] = df["AskVolume_0"]*0
df["Sign"] = df["AskVolume_0"]*0
df["Quote"] = df["AskVolume_0"]*0
df["Quote"].replace(0, "NoBest", inplace = True)
df["Type"] = df["BidPrice_0"]*0

In [23]:
clean_data.iloc[11086:11089,30:]

,BidVolume_7,AskPrice_7,AskVolume_7,BidPrice_8,BidVolume_8,AskPrice_8,AskVolume_8,BidPrice_9,BidVolume_9,AskPrice_9,AskVolume_9,MidPrice,Spread,Time
11086,5.0,138.50,1.0,134.0,5.0,139.45,1.0,133.3,3.0,140.00,1.0,135.275,0.25,27139.225
11087,5.0,139.45,1.0,134.0,5.0,140.00,1.0,133.3,3.0,0.00,0.0,135.275,0.25,27139.228
11088,5.0,139.45,1.0,134.0,5.0,140.00,1.0,133.3,3.0,140.03,1.0,135.275,0.25,27139.230


In [24]:
df.iloc[11087:11089,30:]

,AskPrice_7,AskVolume_7,BidPrice_8,BidVolume_8,AskPrice_8,AskVolume_8,BidPrice_9,BidVolume_9,AskPrice_9,AskVolume_9,Price,Volume,Sign,Quote,Type
11087,0.95,0.0,0.0,0.0,0.55,0.0,0.0,0.0,-140.00,-1.0,0.0,0.0,0.0,NoBest,0.0
11088,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,140.03,1.0,0.0,0.0,0.0,NoBest,0.0


In [4]:
#@title Quote 0
# BidPrice_0 > 0 
con = (df["BidPrice_0"] > 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Limit"
df.loc[a,["Quote"]] = 0
df.loc[a,["Sign"]] = 1
df.loc[a,["Price"]] = clean_data.loc[a]["BidPrice_0"]
df.loc[a,["Volume"]] = clean_data.loc[a]["BidVolume_0"]

# BidPrice_0 < 0 
con = (df["BidPrice_0"] < 0) 
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Market/Cancel"
df.loc[a,["Quote"]] = 0
df.loc[a,["Sign"]] = 1
df.loc[a,["Price"]] = clean_data.loc[a-1]["BidPrice_0"].to_list()
df.loc[a,["Volume"]] = clean_data.loc[a-1]["BidVolume_0"].to_list()

# AskPrice_0 > 0
con = (df["AskPrice_0"] > 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Market/Cancel"
df.loc[a,["Quote"]] = 0
df.loc[a,["Sign"]] = -1
df.loc[a,["Price"]] = clean_data.loc[a-1]["AskPrice_0"].to_list()
df.loc[a,["Volume"]] = clean_data.loc[a-1]["AskVolume_0"].to_list()

# AskPrice_0 < 0
con = (df["AskPrice_0"] < 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Limit"
df.loc[a,["Quote"]] = 0
df.loc[a,["Sign"]] = -1
df.loc[a,["Price"]] = clean_data.loc[a]["AskPrice_0"]
df.loc[a,["Volume"]] = clean_data.loc[a]["AskVolume_0"]

# BidVolume_0 > 0
con = (df["BidPrice_0"] == 0) & (df["BidVolume_0"] > 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Limit"
df.loc[a,["Quote"]] = 0
df.loc[a,["Sign"]] = 1
df.loc[a,["Price"]] = clean_data.loc[a]["BidPrice_0"]
df.loc[a,["Volume"]] = df.loc[a]["BidVolume_0"]

# BidVolume_0 < 0
con = (df["BidPrice_0"] == 0) & (df["BidVolume_0"] < 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Market/Cancel"
df.loc[a,["Quote"]] = 0
df.loc[a,["Sign"]] = 1
df.loc[a,["Price"]] = clean_data.loc[a]["BidPrice_0"]
df.loc[a,["Volume"]] = df.loc[a]["BidVolume_0"] * (-1)

# AskVolume_0 > 0
con = (df["AskPrice_0"] == 0) & (df["AskVolume_0"] > 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Limit"
df.loc[a,["Quote"]] = 0
df.loc[a,["Sign"]] = -1
df.loc[a,["Price"]] = clean_data.loc[a]["AskPrice_0"]
df.loc[a,["Volume"]] = df.loc[a]["AskVolume_0"]

# AskVolume_0 < 0
con = (df["BidPrice_0"] == 0) & (df["AskVolume_0"] < 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Market/Cancel"
df.loc[a,["Quote"]] = 0
df.loc[a,["Sign"]] = -1
df.loc[a,["Price"]] = clean_data.loc[a]["AskPrice_0"]
df.loc[a,["Volume"]] = df.loc[a]["AskVolume_0"]* (-1)

In [5]:
#@title Quote 1
# BidPrice_1 > 0
con = (df["BidPrice_0"] == 0) & (df["BidVolume_0"] == 0) & (df["BidPrice_1"] > 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Limit"
df.loc[a,["Quote"]] = 1
df.loc[a,["Sign"]] = 1
df.loc[a,["Price"]] = clean_data.loc[a]["BidPrice_1"]
df.loc[a,["Volume"]] = clean_data.loc[a]["BidVolume_1"]

# BidPrice_1 < 0
con = (df["BidPrice_0"] == 0) & (df["BidVolume_0"] == 0) & (df["BidPrice_1"] < 0) 
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Market/Cancel"
df.loc[a,["Quote"]] = 1
df.loc[a,["Sign"]] = 1
df.loc[a,["Price"]] = clean_data.loc[a-1]["BidPrice_1"].to_list()
df.loc[a,["Volume"]] = clean_data.loc[a-1]["BidVolume_1"].to_list()

# AskPrice_1 > 0
con = (df["AskPrice_0"] == 0) & (df["AskVolume_0"] == 0) & (df["AskPrice_1"] > 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Market/Cancel"
df.loc[a,["Quote"]] = 1
df.loc[a,["Sign"]] = -1
df.loc[a,["Price"]] = clean_data.loc[a-1]["AskPrice_1"].to_list()
df.loc[a,["Volume"]] = clean_data.loc[a-1]["AskVolume_1"].to_list()

# AskPrice_1 < 0
con = (df["AskPrice_0"] == 0) & (df["AskVolume_0"] == 0) & (df["AskPrice_1"] < 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Limit"
df.loc[a,["Quote"]] = 1
df.loc[a,["Sign"]] = -1
df.loc[a,["Price"]] = clean_data.loc[a]["AskPrice_1"]
df.loc[a,["Volume"]] = clean_data.loc[a]["AskVolume_1"]

# BidVolume_1 > 0
con = (df["BidPrice_0"] == 0) & (df["BidVolume_0"] == 0) & (df["BidPrice_1"] == 0) & (df["BidVolume_1"] >0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Limit"
df.loc[a,["Quote"]] = 1
df.loc[a,["Sign"]] = 1
df.loc[a,["Price"]] = clean_data.loc[a]["BidPrice_1"]
df.loc[a,["Volume"]] = df.loc[a]["BidVolume_1"]

# BidVolume_1 < 0
con = (df["BidPrice_0"] == 0) & (df["BidVolume_0"] == 0) & (df["BidPrice_1"] == 0) & (df["BidVolume_1"] < 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Market/Cancel"
df.loc[a,["Quote"]] = 1
df.loc[a,["Sign"]] = 1
df.loc[a,["Price"]] = clean_data.loc[a]["BidPrice_1"]
df.loc[a,["Volume"]] = df.loc[a]["BidVolume_1"] * (-1)

# AskVolume_1 > 0
con = (df["AskPrice_0"] == 0) & (df["AskVolume_0"] == 0) & (df["AskPrice_1"] == 0) & (df["AskVolume_1"] > 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Limit"
df.loc[a,["Quote"]] = 1
df.loc[a,["Sign"]] = -1
df.loc[a,["Price"]] = clean_data.loc[a]["AskPrice_1"]
df.loc[a,["Volume"]] = df.loc[a]["AskVolume_1"]

# AskVolume_1 < 0
con = (df["AskPrice_0"] == 0) & (df["AskVolume_0"] == 0) & (df["AskPrice_1"] == 0) & (df["AskVolume_1"] < 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Market/Cancel"
df.loc[a,["Quote"]] = 1
df.loc[a,["Sign"]] = -1
df.loc[a,["Price"]] = clean_data.loc[a]["AskPrice_1"]
df.loc[a,["Volume"]] = df.loc[a]["AskVolume_1"]* (-1)

In [6]:
#@title Quote 2
# BidPrice_2 > 0
con = (df["BidPrice_0"] == 0) & (df["BidVolume_0"] == 0) & (df["BidPrice_1"] == 0) & (df["BidVolume_1"] == 0) \
      & (df["BidPrice_2"] > 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Limit"
df.loc[a,["Quote"]] = 2
df.loc[a,["Sign"]] = 1
df.loc[a,["Price"]] = clean_data.loc[a]["BidPrice_2"]
df.loc[a,["Volume"]] = clean_data.loc[a]["BidVolume_2"]

# BidPrice_2 < 0
con = (df["BidPrice_0"] == 0) & (df["BidVolume_0"] == 0) & (df["BidPrice_1"] == 0) & (df["BidVolume_1"] == 0) \
      & (df["BidPrice_2"] < 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Market/Cancel"
df.loc[a,["Quote"]] = 2
df.loc[a,["Sign"]] = 1
df.loc[a,["Price"]] = clean_data.loc[a-1]["BidPrice_2"].to_list()
df.loc[a,["Volume"]] = clean_data.loc[a-1]["BidVolume_2"].to_list()

# AskPrice_2 > 0
con = (df["AskPrice_0"] == 0) & (df["AskVolume_0"] == 0) & (df["AskPrice_1"] == 0) & (df["AskVolume_1"] == 0) \
      & (df["AskPrice_2"] > 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Market/Cancel"
df.loc[a,["Quote"]] = 2
df.loc[a,["Sign"]] = -1
df.loc[a,["Price"]] = clean_data.loc[a-1]["AskPrice_2"].to_list()
df.loc[a,["Volume"]] = clean_data.loc[a-1]["AskVolume_2"].to_list()

# AskPrice_2 < 0
con = (df["AskPrice_0"] == 0) & (df["AskVolume_0"] == 0) & (df["AskPrice_1"] == 0) & (df["AskVolume_1"] == 0) \
      & (df["AskPrice_2"] < 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Limit"
df.loc[a,["Quote"]] = 2
df.loc[a,["Sign"]] = -1
df.loc[a,["Price"]] = clean_data.loc[a]["AskPrice_2"]
df.loc[a,["Volume"]] = clean_data.loc[a]["AskVolume_2"]


# BidVolume_2 > 0
con = (df["BidPrice_0"] == 0) & (df["BidVolume_0"] == 0) & (df["BidPrice_1"] == 0) & (df["BidVolume_1"] == 0) \
      & (df["BidPrice_2"] == 0) & (df["BidVolume_2"] > 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Limit"
df.loc[a,["Quote"]] = 2
df.loc[a,["Sign"]] = 1
df.loc[a,["Price"]] = clean_data.loc[a]["BidPrice_2"]
df.loc[a,["Volume"]] = df.loc[a]["BidVolume_2"]

# BidVolume_2 < 0
con = (df["BidPrice_0"] == 0) & (df["BidVolume_0"] == 0) & (df["BidPrice_1"] == 0) & (df["BidVolume_1"] == 0) \
      & (df["BidPrice_2"] == 0) & (df["BidVolume_2"] < 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Market/Cancel"
df.loc[a,["Quote"]] = 2
df.loc[a,["Sign"]] = 1
df.loc[a,["Price"]] = clean_data.loc[a]["BidPrice_2"]
df.loc[a,["Volume"]] = df.loc[a]["BidVolume_2"] * (-1)

# AskVolume_2 > 0
con = (df["AskPrice_0"] == 0) & (df["AskVolume_0"] == 0) & (df["AskPrice_1"] == 0) & (df["AskVolume_1"] == 0) \
      & (df["AskPrice_2"] == 0) & (df["AskVolume_2"] > 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Limit"
df.loc[a,["Quote"]] = 2
df.loc[a,["Sign"]] = -1
df.loc[a,["Price"]] = clean_data.loc[a]["AskPrice_2"]
df.loc[a,["Volume"]] = df.loc[a]["AskVolume_2"]

# AskVolume_2 < 0
con = (df["AskPrice_0"] == 0) & (df["AskVolume_0"] == 0) & (df["AskPrice_1"] == 0) & (df["AskVolume_1"] == 0) \
      & (df["AskPrice_2"] == 0) & (df["AskVolume_2"] < 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Market/Cancel"
df.loc[a,["Quote"]] = 2
df.loc[a,["Sign"]] = -1
df.loc[a,["Price"]] = clean_data.loc[a]["AskPrice_2"]
df.loc[a,["Volume"]] = df.loc[a]["AskVolume_2"]* (-1)

In [7]:
#@title Quote 3

# BidPrice_3 > 0
con = (df["BidPrice_0"] == 0) & (df["BidVolume_0"] == 0) & (df["BidPrice_1"] == 0) & (df["BidVolume_1"] == 0) \
      & (df["BidPrice_2"] == 0) & (df["BidVolume_2"] == 0) & (df["BidPrice_3"] > 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Limit"
df.loc[a,["Quote"]] = 3
df.loc[a,["Sign"]] = 1
df.loc[a,["Price"]] = clean_data.loc[a]["BidPrice_3"]
df.loc[a,["Volume"]] = clean_data.loc[a]["BidVolume_3"]

# BidPrice_3 < 0
con = (df["BidPrice_0"] == 0) & (df["BidVolume_0"] == 0) & (df["BidPrice_1"] == 0) & (df["BidVolume_1"] == 0) \
      & (df["BidPrice_2"] == 0) & (df["BidVolume_2"] == 0) & (df["BidPrice_3"] < 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Market/Cancel"
df.loc[a,["Quote"]] = 3
df.loc[a,["Sign"]] = 1
df.loc[a,["Price"]] = clean_data.loc[a-1]["BidPrice_3"].to_list()
df.loc[a,["Volume"]] = clean_data.loc[a-1]["BidVolume_3"].to_list()

# AskPrice_3 > 0
con = (df["AskPrice_0"] == 0) & (df["AskVolume_0"] == 0) & (df["AskPrice_1"] == 0) & (df["AskVolume_1"] == 0) \
      & (df["AskPrice_2"] == 0) & (df["AskVolume_2"] == 0) & (df["AskPrice_3"]  > 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Market/Cancel"
df.loc[a,["Quote"]] = 3
df.loc[a,["Sign"]] = -1
df.loc[a,["Price"]] = clean_data.loc[a-1]["AskPrice_3"].to_list()
df.loc[a,["Volume"]] = clean_data.loc[a-1]["AskVolume_3"].to_list()

# AskPrice_3 < 0
con = (df["AskPrice_0"] == 0) & (df["AskVolume_0"] == 0) & (df["AskPrice_1"] == 0) & (df["AskVolume_1"] == 0) \
      & (df["AskPrice_2"] == 0) & (df["AskVolume_2"] == 0) & (df["AskPrice_3"]  < 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Limit"
df.loc[a,["Quote"]] = 3
df.loc[a,["Sign"]] = -1
df.loc[a,["Price"]] = clean_data.loc[a]["AskPrice_3"]
df.loc[a,["Volume"]] = clean_data.loc[a]["AskVolume_3"]


# BidVolume_3 > 0
con = (df["BidPrice_0"] == 0) & (df["BidVolume_0"] == 0) & (df["BidPrice_1"] == 0) & (df["BidVolume_1"] == 0) \
      & (df["BidPrice_2"] == 0) & (df["BidVolume_2"] == 0) & (df["BidPrice_3"] == 0) & (df["BidVolume_3"] > 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Limit"
df.loc[a,["Quote"]] = 3
df.loc[a,["Sign"]] = 1
df.loc[a,["Price"]] = clean_data.loc[a]["BidPrice_3"]
df.loc[a,["Volume"]] = df.loc[a]["BidVolume_3"]

# BidVolume_3 < 0
con = (df["BidPrice_0"] == 0) & (df["BidVolume_0"] == 0) & (df["BidPrice_1"] == 0) & (df["BidVolume_1"] == 0) \
      & (df["BidPrice_2"] == 0) & (df["BidVolume_2"] == 0) & (df["BidPrice_3"] == 0) & (df["BidVolume_3"] < 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Market/Cancel"
df.loc[a,["Quote"]] = 3
df.loc[a,["Sign"]] = 1
df.loc[a,["Price"]] = clean_data.loc[a]["BidPrice_3"]
df.loc[a,["Volume"]] = df.loc[a]["BidVolume_3"] * (-1)

# AskVolume_3 > 0
con = (df["AskPrice_0"] == 0) & (df["AskVolume_0"] == 0) & (df["AskPrice_1"] == 0) & (df["AskVolume_1"] == 0) \
      & (df["AskPrice_2"] == 0) & (df["AskVolume_2"] == 0) & (df["AskPrice_3"]  == 0) & (df["AskVolume_3"] > 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Limit"
df.loc[a,["Quote"]] = 3
df.loc[a,["Sign"]] = -1
df.loc[a,["Price"]] = clean_data.loc[a]["AskPrice_3"]
df.loc[a,["Volume"]] = df.loc[a]["AskVolume_3"]

# AskVolume_3 < 0
con = (df["AskPrice_0"] == 0) & (df["AskVolume_0"] == 0) & (df["AskPrice_1"] == 0) & (df["AskVolume_1"] == 0) \
      & (df["AskPrice_2"] == 0) & (df["AskVolume_2"] == 0) & (df["AskPrice_3"]  == 0) & (df["AskVolume_3"] < 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Market/Cancel"
df.loc[a,["Quote"]] = 3
df.loc[a,["Sign"]] = -1
df.loc[a,["Price"]] = clean_data.loc[a]["AskPrice_3"]
df.loc[a,["Volume"]] = df.loc[a]["AskVolume_3"]* (-1)

In [8]:
#@title Quote 4
# BidPrice_4 > 0
con = (df["BidPrice_0"] == 0) & (df["BidVolume_0"] == 0) & (df["BidPrice_1"] == 0) & (df["BidVolume_1"] == 0) \
      & (df["BidPrice_2"] == 0) & (df["BidVolume_2"] == 0) & (df["BidPrice_3"] == 0) & (df["BidVolume_3"] == 0) \
      & (df["BidPrice_4"] > 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Limit"
df.loc[a,["Quote"]] = 4
df.loc[a,["Sign"]] = 1
df.loc[a,["Price"]] = clean_data.loc[a]["BidPrice_4"]
df.loc[a,["Volume"]] = clean_data.loc[a]["BidVolume_4"]

# BidPrice_4 < 0
con = (df["BidPrice_0"] == 0) & (df["BidVolume_0"] == 0) & (df["BidPrice_1"] == 0) & (df["BidVolume_1"] == 0) \
      & (df["BidPrice_2"] == 0) & (df["BidVolume_2"] == 0) & (df["BidPrice_3"] == 0) & (df["BidVolume_3"] == 0) \
      & (df["BidPrice_4"] < 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Market/Cancel"
df.loc[a,["Quote"]] = 4
df.loc[a,["Sign"]] = 1
df.loc[a,["Price"]] = clean_data.loc[a-1]["BidPrice_4"].to_list()
df.loc[a,["Volume"]] = clean_data.loc[a-1]["BidVolume_4"].to_list()

# AskPrice_4 > 0
con = (df["AskPrice_0"] == 0) & (df["AskVolume_0"] == 0) & (df["AskPrice_1"] == 0) & (df["AskVolume_1"] == 0) \
      & (df["AskPrice_2"] == 0) & (df["AskVolume_2"] == 0) & (df["AskPrice_3"]  == 0) & (df["AskVolume_3"] == 0) \
      & (df["AskPrice_4"] > 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Market/Cancel"
df.loc[a,["Quote"]] = 4
df.loc[a,["Sign"]] = -1
df.loc[a,["Price"]] = clean_data.loc[a-1]["AskPrice_4"].to_list()
df.loc[a,["Volume"]] = clean_data.loc[a-1]["AskVolume_4"].to_list()

# AskPrice_4 < 0
con = (df["AskPrice_0"] == 0) & (df["AskVolume_0"] == 0) & (df["AskPrice_1"] == 0) & (df["AskVolume_1"] == 0) \
      & (df["AskPrice_2"] == 0) & (df["AskVolume_2"] == 0) & (df["AskPrice_3"]  == 0) & (df["AskVolume_3"] == 0) \
      & (df["AskPrice_4"] < 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Limit"
df.loc[a,["Quote"]] = 4
df.loc[a,["Sign"]] = -1
df.loc[a,["Price"]] = clean_data.loc[a]["AskPrice_4"]
df.loc[a,["Volume"]] = clean_data.loc[a]["AskVolume_4"]


# BidVolume_4 > 0
con = (df["BidPrice_0"] == 0) & (df["BidVolume_0"] == 0) & (df["BidPrice_1"] == 0) & (df["BidVolume_1"] == 0) \
      & (df["BidPrice_2"] == 0) & (df["BidVolume_2"] == 0) & (df["BidPrice_3"] == 0) & (df["BidVolume_3"] == 0) \
      & (df["BidPrice_4"] == 0) & (df["BidVolume_4"] > 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Limit"
df.loc[a,["Quote"]] = 4
df.loc[a,["Sign"]] = 1
df.loc[a,["Price"]] = clean_data.loc[a]["BidPrice_4"]
df.loc[a,["Volume"]] = df.loc[a]["BidVolume_4"]

# BidVolume_4 < 0
con = (df["BidPrice_0"] == 0) & (df["BidVolume_0"] == 0) & (df["BidPrice_1"] == 0) & (df["BidVolume_1"] == 0) \
      & (df["BidPrice_2"] == 0) & (df["BidVolume_2"] == 0) & (df["BidPrice_3"] == 0) & (df["BidVolume_3"] == 0) \
      & (df["BidPrice_4"] == 0) & (df["BidVolume_4"] < 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Market/Cancel"
df.loc[a,["Quote"]] = 4
df.loc[a,["Sign"]] = 1
df.loc[a,["Price"]] = clean_data.loc[a]["BidPrice_4"]
df.loc[a,["Volume"]] = df.loc[a]["BidVolume_4"] * (-1)

# AskVolume_4 > 0
con = (df["AskPrice_0"] == 0) & (df["AskVolume_0"] == 0) & (df["AskPrice_1"] == 0) & (df["AskVolume_1"] == 0) \
      & (df["AskPrice_2"] == 0) & (df["AskVolume_2"] == 0) & (df["AskPrice_3"]  == 0) & (df["AskVolume_3"] == 0) \
      & (df["AskPrice_4"] == 0) & (df["AskVolume_4"] > 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Limit"
df.loc[a,["Quote"]] = 4
df.loc[a,["Sign"]] = -1
df.loc[a,["Price"]] = clean_data.loc[a]["AskPrice_4"]
df.loc[a,["Volume"]] = df.loc[a]["AskVolume_4"]

# AskVolume_4 < 0
con = (df["AskPrice_0"] == 0) & (df["AskVolume_0"] == 0) & (df["AskPrice_1"] == 0) & (df["AskVolume_1"] == 0) \
      & (df["AskPrice_2"] == 0) & (df["AskVolume_2"] == 0) & (df["AskPrice_3"]  == 0) & (df["AskVolume_3"] == 0) \
      & (df["AskPrice_4"] == 0) & (df["AskVolume_4"] < 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Market/Cancel"
df.loc[a,["Quote"]] = 4
df.loc[a,["Sign"]] = -1
df.loc[a,["Price"]] = clean_data.loc[a]["AskPrice_4"]
df.loc[a,["Volume"]] = df.loc[a]["AskVolume_4"]* (-1)

In [9]:
#@title Quote 5

# BidPrice_5 > 0
con = (df["BidPrice_0"] == 0) & (df["BidVolume_0"] == 0) & (df["BidPrice_1"] == 0) & (df["BidVolume_1"] == 0) \
      & (df["BidPrice_2"] == 0) & (df["BidVolume_2"] == 0) & (df["BidPrice_3"] == 0) & (df["BidVolume_3"] == 0) \
      & (df["BidPrice_4"] == 0) & (df["BidVolume_4"] == 0) & (df["BidPrice_5"] > 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Limit"
df.loc[a,["Quote"]] = 5
df.loc[a,["Sign"]] = 1
df.loc[a,["Price"]] = clean_data.loc[a]["BidPrice_5"]
df.loc[a,["Volume"]] = clean_data.loc[a]["BidVolume_5"]

# BidPrice_5 < 0
con = (df["BidPrice_0"] == 0) & (df["BidVolume_0"] == 0) & (df["BidPrice_1"] == 0) & (df["BidVolume_1"] == 0) \
      & (df["BidPrice_2"] == 0) & (df["BidVolume_2"] == 0) & (df["BidPrice_3"] == 0) & (df["BidVolume_3"] == 0) \
      & (df["BidPrice_4"] == 0) & (df["BidVolume_4"] == 0) & (df["BidPrice_5"] < 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Market/Cancel"
df.loc[a,["Quote"]] = 5
df.loc[a,["Sign"]] = 1
df.loc[a,["Price"]] = clean_data.loc[a-1]["BidPrice_5"].to_list()
df.loc[a,["Volume"]] = clean_data.loc[a-1]["BidVolume_5"].to_list()

# AskPrice_5 > 0
con = (df["AskPrice_0"] == 0) & (df["AskVolume_0"] == 0) & (df["AskPrice_1"] == 0) & (df["AskVolume_1"] == 0) \
      & (df["AskPrice_2"] == 0) & (df["AskVolume_2"] == 0) & (df["AskPrice_3"]  == 0) & (df["AskVolume_3"] == 0) \
      & (df["AskPrice_4"] == 0) & (df["AskVolume_4"] == 0) & (df["AskPrice_5"] > 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Market/Cancel"
df.loc[a,["Quote"]] = 5
df.loc[a,["Sign"]] = -1
df.loc[a,["Price"]] = clean_data.loc[a-1]["AskPrice_5"].to_list()
df.loc[a,["Volume"]] = clean_data.loc[a-1]["AskVolume_5"].to_list()

# AskPrice_5 < 0
con = (df["AskPrice_0"] == 0) & (df["AskVolume_0"] == 0) & (df["AskPrice_1"] == 0) & (df["AskVolume_1"] == 0) \
      & (df["AskPrice_2"] == 0) & (df["AskVolume_2"] == 0) & (df["AskPrice_3"]  == 0) & (df["AskVolume_3"] == 0) \
      & (df["AskPrice_4"] == 0) & (df["AskVolume_4"] == 0) & (df["AskPrice_5"] < 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Limit"
df.loc[a,["Quote"]] = 5
df.loc[a,["Sign"]] = -1
df.loc[a,["Price"]] = clean_data.loc[a]["AskPrice_5"]
df.loc[a,["Volume"]] = clean_data.loc[a]["AskVolume_5"]


# BidVolume_5 > 0
con = (df["BidPrice_0"] == 0) & (df["BidVolume_0"] == 0) & (df["BidPrice_1"] == 0) & (df["BidVolume_1"] == 0) \
      & (df["BidPrice_2"] == 0) & (df["BidVolume_2"] == 0) & (df["BidPrice_3"] == 0) & (df["BidVolume_3"] == 0) \
      & (df["BidPrice_4"] == 0) & (df["BidVolume_4"] == 0) & (df["BidPrice_5"] == 0) & (df["BidVolume_5"] > 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Limit"
df.loc[a,["Quote"]] = 5
df.loc[a,["Sign"]] = 1
df.loc[a,["Price"]] = clean_data.loc[a]["BidPrice_5"]
df.loc[a,["Volume"]] = df.loc[a]["BidVolume_5"]

# BidVolume_5 < 0
con = (df["BidPrice_0"] == 0) & (df["BidVolume_0"] == 0) & (df["BidPrice_1"] == 0) & (df["BidVolume_1"] == 0) \
      & (df["BidPrice_2"] == 0) & (df["BidVolume_2"] == 0) & (df["BidPrice_3"] == 0) & (df["BidVolume_3"] == 0) \
      & (df["BidPrice_4"] == 0) & (df["BidVolume_4"] == 0) & (df["BidPrice_5"] == 0) & (df["BidVolume_5"] < 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Market/Cancel"
df.loc[a,["Quote"]] = 5
df.loc[a,["Sign"]] = 1
df.loc[a,["Price"]] = clean_data.loc[a]["BidPrice_5"]
df.loc[a,["Volume"]] = df.loc[a]["BidVolume_5"] * (-1)

# AskVolume_5 > 0
con = (df["AskPrice_0"] == 0) & (df["AskVolume_0"] == 0) & (df["AskPrice_1"] == 0) & (df["AskVolume_1"] == 0) \
      & (df["AskPrice_2"] == 0) & (df["AskVolume_2"] == 0) & (df["AskPrice_3"]  == 0) & (df["AskVolume_3"] == 0) \
      & (df["AskPrice_4"] == 0) & (df["AskVolume_4"] == 0) & (df["AskPrice_5"] == 0) & (df["AskVolume_5"] > 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Limit"
df.loc[a,["Quote"]] = 5
df.loc[a,["Sign"]] = -1
df.loc[a,["Price"]] = clean_data.loc[a]["AskPrice_5"]
df.loc[a,["Volume"]] = df.loc[a]["AskVolume_5"]

# AskVolume_5 < 0
con = (df["AskPrice_0"] == 0) & (df["AskVolume_0"] == 0) & (df["AskPrice_1"] == 0) & (df["AskVolume_1"] == 0) \
      & (df["AskPrice_2"] == 0) & (df["AskVolume_2"] == 0) & (df["AskPrice_3"]  == 0) & (df["AskVolume_3"] == 0) \
      & (df["AskPrice_4"] == 0) & (df["AskVolume_4"] == 0) & (df["AskPrice_5"] == 0) & (df["AskVolume_5"] < 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Market/Cancel"
df.loc[a,["Quote"]] = 5
df.loc[a,["Sign"]] = -1
df.loc[a,["Price"]] = clean_data.loc[a]["AskPrice_5"]
df.loc[a,["Volume"]] = df.loc[a]["AskVolume_5"]* (-1)

In [10]:
#@title Quote 6

# BidPrice_6 > 0
con = (df["BidPrice_0"] == 0) & (df["BidVolume_0"] == 0) & (df["BidPrice_1"] == 0) & (df["BidVolume_1"] == 0) \
      & (df["BidPrice_2"] == 0) & (df["BidVolume_2"] == 0) & (df["BidPrice_3"] == 0) & (df["BidVolume_3"] == 0) \
      & (df["BidPrice_4"] == 0) & (df["BidVolume_4"] == 0) & (df["BidPrice_5"] == 0) & (df["BidVolume_5"] == 0) \
      & (df["BidPrice_6"] > 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Limit"
df.loc[a,["Quote"]] = 6
df.loc[a,["Sign"]] = 1
df.loc[a,["Price"]] = clean_data.loc[a]["BidPrice_6"]
df.loc[a,["Volume"]] = clean_data.loc[a]["BidVolume_6"]

# BidPrice_6 < 0
con = (df["BidPrice_0"] == 0) & (df["BidVolume_0"] == 0) & (df["BidPrice_1"] == 0) & (df["BidVolume_1"] == 0) \
      & (df["BidPrice_2"] == 0) & (df["BidVolume_2"] == 0) & (df["BidPrice_3"] == 0) & (df["BidVolume_3"] == 0) \
      & (df["BidPrice_4"] == 0) & (df["BidVolume_4"] == 0) & (df["BidPrice_5"] == 0) & (df["BidVolume_5"] == 0) \
      & (df["BidPrice_6"] < 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Market/Cancel"
df.loc[a,["Quote"]] = 6
df.loc[a,["Sign"]] = 1
df.loc[a,["Price"]] = clean_data.loc[a-1]["BidPrice_6"].to_list()
df.loc[a,["Volume"]] = clean_data.loc[a-1]["BidVolume_6"].to_list()

# AskPrice_6 > 0
con = (df["AskPrice_0"] == 0) & (df["AskVolume_0"] == 0) & (df["AskPrice_1"] == 0) & (df["AskVolume_1"] == 0) \
      & (df["AskPrice_2"] == 0) & (df["AskVolume_2"] == 0) & (df["AskPrice_3"]  == 0) & (df["AskVolume_3"] == 0) \
      & (df["AskPrice_4"] == 0) & (df["AskVolume_4"] == 0) & (df["AskPrice_5"] == 0) & (df["AskVolume_5"] == 0) \
      & (df["AskPrice_6"] > 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Market/Cancel"
df.loc[a,["Quote"]] = 6
df.loc[a,["Sign"]] = -1
df.loc[a,["Price"]] = clean_data.loc[a-1]["AskPrice_6"].to_list()
df.loc[a,["Volume"]] = clean_data.loc[a-1]["AskVolume_6"].to_list()

# AskPrice_6 < 0
con = (df["AskPrice_0"] == 0) & (df["AskVolume_0"] == 0) & (df["AskPrice_1"] == 0) & (df["AskVolume_1"] == 0) \
      & (df["AskPrice_2"] == 0) & (df["AskVolume_2"] == 0) & (df["AskPrice_3"]  == 0) & (df["AskVolume_3"] == 0) \
      & (df["AskPrice_4"] == 0) & (df["AskVolume_4"] == 0) & (df["AskPrice_5"] == 0) & (df["AskVolume_5"] == 0) \
      & (df["AskPrice_6"] < 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Limit"
df.loc[a,["Quote"]] = 6
df.loc[a,["Sign"]] = -1
df.loc[a,["Price"]] = clean_data.loc[a]["AskPrice_6"]
df.loc[a,["Volume"]] = clean_data.loc[a]["AskVolume_6"]


# BidVolume_6 > 0
con = (df["BidPrice_0"] == 0) & (df["BidVolume_0"] == 0) & (df["BidPrice_1"] == 0) & (df["BidVolume_1"] == 0) \
      & (df["BidPrice_2"] == 0) & (df["BidVolume_2"] == 0) & (df["BidPrice_3"] == 0) & (df["BidVolume_3"] == 0) \
      & (df["BidPrice_4"] == 0) & (df["BidVolume_4"] == 0) & (df["BidPrice_5"] == 0) & (df["BidVolume_5"] == 0) \
      & (df["BidPrice_6"] == 0) & (df["BidVolume_6"] > 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Limit"
df.loc[a,["Quote"]] = 6
df.loc[a,["Sign"]] = 1
df.loc[a,["Price"]] = clean_data.loc[a]["BidPrice_6"]
df.loc[a,["Volume"]] = df.loc[a]["BidVolume_6"]

# BidVolume_6 < 0
con = (df["BidPrice_0"] == 0) & (df["BidVolume_0"] == 0) & (df["BidPrice_1"] == 0) & (df["BidVolume_1"] == 0) \
      & (df["BidPrice_2"] == 0) & (df["BidVolume_2"] == 0) & (df["BidPrice_3"] == 0) & (df["BidVolume_3"] == 0) \
      & (df["BidPrice_4"] == 0) & (df["BidVolume_4"] == 0) & (df["BidPrice_5"] == 0) & (df["BidVolume_5"] == 0) \
      & (df["BidPrice_6"] == 0) & (df["BidVolume_6"] < 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Market/Cancel"
df.loc[a,["Quote"]] = 6
df.loc[a,["Sign"]] = 1
df.loc[a,["Price"]] = clean_data.loc[a]["BidPrice_6"]
df.loc[a,["Volume"]] = df.loc[a]["BidVolume_6"] * (-1)

# AskVolume_6 > 0
con = (df["AskPrice_0"] == 0) & (df["AskVolume_0"] == 0) & (df["AskPrice_1"] == 0) & (df["AskVolume_1"] == 0) \
      & (df["AskPrice_2"] == 0) & (df["AskVolume_2"] == 0) & (df["AskPrice_3"]  == 0) & (df["AskVolume_3"] == 0) \
      & (df["AskPrice_4"] == 0) & (df["AskVolume_4"] == 0) & (df["AskPrice_5"] == 0) & (df["AskVolume_5"] == 0) \
      & (df["AskPrice_6"] == 0) & (df["AskVolume_6"] > 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Limit"
df.loc[a,["Quote"]] = 6
df.loc[a,["Sign"]] = -1
df.loc[a,["Price"]] = clean_data.loc[a]["AskPrice_6"]
df.loc[a,["Volume"]] = df.loc[a]["AskVolume_6"]

# AskVolume_6 < 0
con = (df["AskPrice_0"] == 0) & (df["AskVolume_0"] == 0) & (df["AskPrice_1"] == 0) & (df["AskVolume_1"] == 0) \
      & (df["AskPrice_2"] == 0) & (df["AskVolume_2"] == 0) & (df["AskPrice_3"]  == 0) & (df["AskVolume_3"] == 0) \
      & (df["AskPrice_4"] == 0) & (df["AskVolume_4"] == 0) & (df["AskPrice_5"] == 0) & (df["AskVolume_5"] == 0) \
      & (df["AskPrice_6"] == 0) & (df["AskVolume_6"] < 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Market/Cancel"
df.loc[a,["Quote"]] = 6
df.loc[a,["Sign"]] = -1
df.loc[a,["Price"]] = clean_data.loc[a]["AskPrice_6"]
df.loc[a,["Volume"]] = df.loc[a]["AskVolume_6"]* (-1)

In [11]:
#@title Quote 7

# BidPrice_7 > 0
con = (df["BidPrice_0"] == 0) & (df["BidVolume_0"] == 0) & (df["BidPrice_1"] == 0) & (df["BidVolume_1"] == 0) \
      & (df["BidPrice_2"] == 0) & (df["BidVolume_2"] == 0) & (df["BidPrice_3"] == 0) & (df["BidVolume_3"] == 0) \
      & (df["BidPrice_4"] == 0) & (df["BidVolume_4"] == 0) & (df["BidPrice_5"] == 0) & (df["BidVolume_5"] == 0) \
      & (df["BidPrice_6"] == 0) & (df["BidVolume_6"] == 0) & (df["BidPrice_7"] > 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Limit"
df.loc[a,["Quote"]] = 7
df.loc[a,["Sign"]] = 1
df.loc[a,["Price"]] = clean_data.loc[a]["BidPrice_7"]
df.loc[a,["Volume"]] = clean_data.loc[a]["BidVolume_7"]

# BidPrice_7 < 0
con = (df["BidPrice_0"] == 0) & (df["BidVolume_0"] == 0) & (df["BidPrice_1"] == 0) & (df["BidVolume_1"] == 0) \
      & (df["BidPrice_2"] == 0) & (df["BidVolume_2"] == 0) & (df["BidPrice_3"] == 0) & (df["BidVolume_3"] == 0) \
      & (df["BidPrice_4"] == 0) & (df["BidVolume_4"] == 0) & (df["BidPrice_5"] == 0) & (df["BidVolume_5"] == 0) \
      & (df["BidPrice_6"] == 0) & (df["BidVolume_6"] == 0) & (df["BidPrice_7"] < 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Market/Cancel"
df.loc[a,["Quote"]] = 7
df.loc[a,["Sign"]] = 1
df.loc[a,["Price"]] = clean_data.loc[a-1]["BidPrice_7"].to_list()
df.loc[a,["Volume"]] = clean_data.loc[a-1]["BidVolume_7"].to_list()

# AskPrice_7 > 0
con = (df["AskPrice_0"] == 0) & (df["AskVolume_0"] == 0) & (df["AskPrice_1"] == 0) & (df["AskVolume_1"] == 0) \
      & (df["AskPrice_2"] == 0) & (df["AskVolume_2"] == 0) & (df["AskPrice_3"]  == 0) & (df["AskVolume_3"] == 0) \
      & (df["AskPrice_4"] == 0) & (df["AskVolume_4"] == 0) & (df["AskPrice_5"] == 0) & (df["AskVolume_5"] == 0) \
      & (df["AskPrice_6"] == 0) & (df["AskVolume_6"] == 0) & (df["AskPrice_7"] > 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Market/Cancel"
df.loc[a,["Quote"]] = 7
df.loc[a,["Sign"]] = -1
df.loc[a,["Price"]] = clean_data.loc[a-1]["AskPrice_7"].to_list()
df.loc[a,["Volume"]] = clean_data.loc[a-1]["AskVolume_7"].to_list()

# AskPrice_7 < 0
con = (df["AskPrice_0"] == 0) & (df["AskVolume_0"] == 0) & (df["AskPrice_1"] == 0) & (df["AskVolume_1"] == 0) \
      & (df["AskPrice_2"] == 0) & (df["AskVolume_2"] == 0) & (df["AskPrice_3"]  == 0) & (df["AskVolume_3"] == 0) \
      & (df["AskPrice_4"] == 0) & (df["AskVolume_4"] == 0) & (df["AskPrice_5"] == 0) & (df["AskVolume_5"] == 0) \
      & (df["AskPrice_6"] == 0) & (df["AskVolume_6"] == 0) & (df["AskPrice_7"] < 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Limit"
df.loc[a,["Quote"]] = 7
df.loc[a,["Sign"]] = -1
df.loc[a,["Price"]] = clean_data.loc[a]["AskPrice_7"]
df.loc[a,["Volume"]] = clean_data.loc[a]["AskVolume_7"]


# BidVolume_7 > 0
con = (df["BidPrice_0"] == 0) & (df["BidVolume_0"] == 0) & (df["BidPrice_1"] == 0) & (df["BidVolume_1"] == 0) \
      & (df["BidPrice_2"] == 0) & (df["BidVolume_2"] == 0) & (df["BidPrice_3"] == 0) & (df["BidVolume_3"] == 0) \
      & (df["BidPrice_4"] == 0) & (df["BidVolume_4"] == 0) & (df["BidPrice_5"] == 0) & (df["BidVolume_5"] == 0) \
      & (df["BidPrice_6"] == 0) & (df["BidVolume_6"] == 0) & (df["BidPrice_7"] == 0) & (df["BidVolume_7"] > 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Limit"
df.loc[a,["Quote"]] = 7
df.loc[a,["Sign"]] = 1
df.loc[a,["Price"]] = clean_data.loc[a]["BidPrice_7"]
df.loc[a,["Volume"]] = df.loc[a]["BidVolume_7"]

# BidVolume_7 < 0
con = (df["BidPrice_0"] == 0) & (df["BidVolume_0"] == 0) & (df["BidPrice_1"] == 0) & (df["BidVolume_1"] == 0) \
      & (df["BidPrice_2"] == 0) & (df["BidVolume_2"] == 0) & (df["BidPrice_3"] == 0) & (df["BidVolume_3"] == 0) \
      & (df["BidPrice_4"] == 0) & (df["BidVolume_4"] == 0) & (df["BidPrice_5"] == 0) & (df["BidVolume_5"] == 0) \
      & (df["BidPrice_6"] == 0) & (df["BidVolume_6"] == 0) & (df["BidPrice_7"] == 0) & (df["BidVolume_7"] < 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Market/Cancel"
df.loc[a,["Quote"]] = 7
df.loc[a,["Sign"]] = 1
df.loc[a,["Price"]] = clean_data.loc[a]["BidPrice_7"]
df.loc[a,["Volume"]] = df.loc[a]["BidVolume_7"] * (-1)

# AskVolume_7 > 0
con = (df["AskPrice_0"] == 0) & (df["AskVolume_0"] == 0) & (df["AskPrice_1"] == 0) & (df["AskVolume_1"] == 0) \
      & (df["AskPrice_2"] == 0) & (df["AskVolume_2"] == 0) & (df["AskPrice_3"]  == 0) & (df["AskVolume_3"] == 0) \
      & (df["AskPrice_4"] == 0) & (df["AskVolume_4"] == 0) & (df["AskPrice_5"] == 0) & (df["AskVolume_5"] == 0) \
      & (df["AskPrice_6"] == 0) & (df["AskVolume_6"] == 0) & (df["AskPrice_7"] == 0) & (df["AskVolume_7"] > 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Limit"
df.loc[a,["Quote"]] = 7
df.loc[a,["Sign"]] = -1
df.loc[a,["Price"]] = clean_data.loc[a]["AskPrice_7"]
df.loc[a,["Volume"]] = df.loc[a]["AskVolume_7"]

# AskVolume_7 < 0
con = (df["AskPrice_0"] == 0) & (df["AskVolume_0"] == 0) & (df["AskPrice_1"] == 0) & (df["AskVolume_1"] == 0) \
      & (df["AskPrice_2"] == 0) & (df["AskVolume_2"] == 0) & (df["AskPrice_3"]  == 0) & (df["AskVolume_3"] == 0) \
      & (df["AskPrice_4"] == 0) & (df["AskVolume_4"] == 0) & (df["AskPrice_5"] == 0) & (df["AskVolume_5"] == 0) \
      & (df["AskPrice_6"] == 0) & (df["AskVolume_6"] == 0) & (df["AskPrice_7"] == 0) & (df["AskVolume_7"] < 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Market/Cancel"
df.loc[a,["Quote"]] = 7
df.loc[a,["Sign"]] = -1
df.loc[a,["Price"]] = clean_data.loc[a]["AskPrice_7"]
df.loc[a,["Volume"]] = df.loc[a]["AskVolume_7"]* (-1)

In [12]:
#@title Quote 8

# BidPrice_8 > 0
con = (df["BidPrice_0"] == 0) & (df["BidVolume_0"] == 0) & (df["BidPrice_1"] == 0) & (df["BidVolume_1"] == 0) \
      & (df["BidPrice_2"] == 0) & (df["BidVolume_2"] == 0) & (df["BidPrice_3"] == 0) & (df["BidVolume_3"] == 0) \
      & (df["BidPrice_4"] == 0) & (df["BidVolume_4"] == 0) & (df["BidPrice_5"] == 0) & (df["BidVolume_5"] == 0) \
      & (df["BidPrice_6"] == 0) & (df["BidVolume_6"] == 0) & (df["BidPrice_7"] == 0) & (df["BidVolume_7"] == 0) \
      & (df["BidPrice_8"] > 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Limit"
df.loc[a,["Quote"]] = 8
df.loc[a,["Sign"]] = 1
df.loc[a,["Price"]] = clean_data.loc[a]["BidPrice_8"]
df.loc[a,["Volume"]] = clean_data.loc[a]["BidVolume_8"]

# BidPrice_8 < 0
con = (df["BidPrice_0"] == 0) & (df["BidVolume_0"] == 0) & (df["BidPrice_1"] == 0) & (df["BidVolume_1"] == 0) \
      & (df["BidPrice_2"] == 0) & (df["BidVolume_2"] == 0) & (df["BidPrice_3"] == 0) & (df["BidVolume_3"] == 0) \
      & (df["BidPrice_4"] == 0) & (df["BidVolume_4"] == 0) & (df["BidPrice_5"] == 0) & (df["BidVolume_5"] == 0) \
      & (df["BidPrice_6"] == 0) & (df["BidVolume_6"] == 0) & (df["BidPrice_7"] == 0) & (df["BidVolume_7"] == 0) \
      & (df["BidPrice_8"] < 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Market/Cancel"
df.loc[a,["Quote"]] = 8
df.loc[a,["Sign"]] = 1
df.loc[a,["Price"]] = clean_data.loc[a-1]["BidPrice_8"].to_list()
df.loc[a,["Volume"]] = clean_data.loc[a-1]["BidVolume_8"].to_list()

# AskPrice_8 > 0
con = (df["AskPrice_0"] == 0) & (df["AskVolume_0"] == 0) & (df["AskPrice_1"] == 0) & (df["AskVolume_1"] == 0) \
      & (df["AskPrice_2"] == 0) & (df["AskVolume_2"] == 0) & (df["AskPrice_3"] == 0) & (df["AskVolume_3"] == 0) \
      & (df["AskPrice_4"] == 0) & (df["AskVolume_4"] == 0) & (df["AskPrice_5"] == 0) & (df["AskVolume_5"] == 0) \
      & (df["AskPrice_6"] == 0) & (df["AskVolume_6"] == 0) & (df["AskPrice_7"] == 0) & (df["AskVolume_7"] == 0) \
      & (df["AskPrice_8"] > 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Market/Cancel"
df.loc[a,["Quote"]] = 8
df.loc[a,["Sign"]] = -1
df.loc[a,["Price"]] = clean_data.loc[a-1]["AskPrice_8"].to_list()
df.loc[a,["Volume"]] = clean_data.loc[a-1]["AskVolume_8"].to_list()

# AskPrice_8 < 0
con = (df["AskPrice_0"] == 0) & (df["AskVolume_0"] == 0) & (df["AskPrice_1"] == 0) & (df["AskVolume_1"] == 0) \
      & (df["AskPrice_2"] == 0) & (df["AskVolume_2"] == 0) & (df["AskPrice_3"] == 0) & (df["AskVolume_3"] == 0) \
      & (df["AskPrice_4"] == 0) & (df["AskVolume_4"] == 0) & (df["AskPrice_5"] == 0) & (df["AskVolume_5"] == 0) \
      & (df["AskPrice_6"] == 0) & (df["AskVolume_6"] == 0) & (df["AskPrice_7"] == 0) & (df["AskVolume_7"] == 0) \
      & (df["AskPrice_8"] < 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Limit"
df.loc[a,["Quote"]] = 8
df.loc[a,["Sign"]] = -1
df.loc[a,["Price"]] = clean_data.loc[a]["AskPrice_8"]
df.loc[a,["Volume"]] = clean_data.loc[a]["AskVolume_8"]


# BidVolume_8 > 0
con = (df["BidPrice_0"] == 0) & (df["BidVolume_0"] == 0) & (df["BidPrice_1"] == 0) & (df["BidVolume_1"] == 0) \
      & (df["BidPrice_2"] == 0) & (df["BidVolume_2"] == 0) & (df["BidPrice_3"] == 0) & (df["BidVolume_3"] == 0) \
      & (df["BidPrice_4"] == 0) & (df["BidVolume_4"] == 0) & (df["BidPrice_5"] == 0) & (df["BidVolume_5"] == 0) \
      & (df["BidPrice_6"] == 0) & (df["BidVolume_6"] == 0) & (df["BidPrice_7"] == 0) & (df["BidVolume_7"] == 0) \
      & (df["BidPrice_8"] == 0) & (df["BidVolume_8"] > 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Limit"
df.loc[a,["Quote"]] = 8
df.loc[a,["Sign"]] = 1
df.loc[a,["Price"]] = clean_data.loc[a]["BidPrice_8"]
df.loc[a,["Volume"]] = df.loc[a]["BidVolume_8"]

# BidVolume_8 < 0
con = (df["BidPrice_0"] == 0) & (df["BidVolume_0"] == 0) & (df["BidPrice_1"] == 0) & (df["BidVolume_1"] == 0) \
      & (df["BidPrice_2"] == 0) & (df["BidVolume_2"] == 0) & (df["BidPrice_3"] == 0) & (df["BidVolume_3"] == 0) \
      & (df["BidPrice_4"] == 0) & (df["BidVolume_4"] == 0) & (df["BidPrice_5"] == 0) & (df["BidVolume_5"] == 0) \
      & (df["BidPrice_6"] == 0) & (df["BidVolume_6"] == 0) & (df["BidPrice_7"] == 0) & (df["BidVolume_7"] == 0) \
      & (df["BidPrice_8"] == 0) & (df["BidVolume_8"] < 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Market/Cancel"
df.loc[a,["Quote"]] = 8
df.loc[a,["Sign"]] = 1
df.loc[a,["Price"]] = clean_data.loc[a]["BidPrice_8"]
df.loc[a,["Volume"]] = df.loc[a]["BidVolume_8"] * (-1)

# AskVolume_8 > 0
con = (df["AskPrice_0"] == 0) & (df["AskVolume_0"] == 0) & (df["AskPrice_1"] == 0) & (df["AskVolume_1"] == 0) \
      & (df["AskPrice_2"] == 0) & (df["AskVolume_2"] == 0) & (df["AskPrice_3"] == 0) & (df["AskVolume_3"] == 0) \
      & (df["AskPrice_4"] == 0) & (df["AskVolume_4"] == 0) & (df["AskPrice_5"] == 0) & (df["AskVolume_5"] == 0) \
      & (df["AskPrice_6"] == 0) & (df["AskVolume_6"] == 0) & (df["AskPrice_7"] == 0) & (df["AskVolume_7"] == 0) \
      & (df["AskPrice_8"] == 0) & (df["AskVolume_8"] > 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Limit"
df.loc[a,["Quote"]] = 8
df.loc[a,["Sign"]] = -1
df.loc[a,["Price"]] = clean_data.loc[a]["AskPrice_8"]
df.loc[a,["Volume"]] = df.loc[a]["AskVolume_8"]

# AskVolume_8 < 0
con = (df["AskPrice_0"] == 0) & (df["AskVolume_0"] == 0) & (df["AskPrice_1"] == 0) & (df["AskVolume_1"] == 0) \
      & (df["AskPrice_2"] == 0) & (df["AskVolume_2"] == 0) & (df["AskPrice_3"] == 0) & (df["AskVolume_3"] == 0) \
      & (df["AskPrice_4"] == 0) & (df["AskVolume_4"] == 0) & (df["AskPrice_5"] == 0) & (df["AskVolume_5"] == 0) \
      & (df["AskPrice_6"] == 0) & (df["AskVolume_6"] == 0) & (df["AskPrice_7"] == 0) & (df["AskVolume_7"] == 0) \
      & (df["AskPrice_8"] == 0) & (df["AskVolume_8"] < 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Market/Cancel"
df.loc[a,["Quote"]] = 8
df.loc[a,["Sign"]] = -1
df.loc[a,["Price"]] = clean_data.loc[a]["AskPrice_8"]
df.loc[a,["Volume"]] = df.loc[a]["AskVolume_8"]* (-1)

In [13]:
#@title Quote 9

# BidPrice_9 > 0
con = (df["BidPrice_0"] == 0) & (df["BidVolume_0"] == 0) & (df["BidPrice_1"] == 0) & (df["BidVolume_1"] == 0) \
      & (df["BidPrice_2"] == 0) & (df["BidVolume_2"] == 0) & (df["BidPrice_3"] == 0) & (df["BidVolume_3"] == 0) \
      & (df["BidPrice_4"] == 0) & (df["BidVolume_4"] == 0) & (df["BidPrice_5"] == 0) & (df["BidVolume_5"] == 0) \
      & (df["BidPrice_6"] == 0) & (df["BidVolume_6"] == 0) & (df["BidPrice_7"] == 0) & (df["BidVolume_7"] == 0) \
      & (df["BidPrice_8"] == 0) & (df["BidVolume_8"] == 0) & (df["BidPrice_9"] > 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Limit"
df.loc[a,["Quote"]] = 9
df.loc[a,["Sign"]] = 1
df.loc[a,["Price"]] = clean_data.loc[a]["BidPrice_9"]
df.loc[a,["Volume"]] = clean_data.loc[a]["BidVolume_9"]

# BidPrice_9 < 0
con = (df["BidPrice_0"] == 0) & (df["BidVolume_0"] == 0) & (df["BidPrice_1"] == 0) & (df["BidVolume_1"] == 0) \
      & (df["BidPrice_2"] == 0) & (df["BidVolume_2"] == 0) & (df["BidPrice_3"] == 0) & (df["BidVolume_3"] == 0) \
      & (df["BidPrice_4"] == 0) & (df["BidVolume_4"] == 0) & (df["BidPrice_5"] == 0) & (df["BidVolume_5"] == 0) \
      & (df["BidPrice_6"] == 0) & (df["BidVolume_6"] == 0) & (df["BidPrice_7"] == 0) & (df["BidVolume_7"] == 0) \
      & (df["BidPrice_8"] == 0) & (df["BidVolume_8"] == 0) & (df["BidPrice_9"] < 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Market/Cancel"
df.loc[a,["Quote"]] = 9
df.loc[a,["Sign"]] = 1
df.loc[a,["Price"]] = clean_data.loc[a-1]["BidPrice_9"].to_list()
df.loc[a,["Volume"]] = clean_data.loc[a-1]["BidVolume_9"].to_list()

# AskPrice_9 > 0
con = (df["AskPrice_0"] == 0) & (df["AskVolume_0"] == 0) & (df["AskPrice_1"] == 0) & (df["AskVolume_1"] == 0) \
      & (df["AskPrice_2"] == 0) & (df["AskVolume_2"] == 0) & (df["AskPrice_3"] == 0) & (df["AskVolume_3"] == 0) \
      & (df["AskPrice_4"] == 0) & (df["AskVolume_4"] == 0) & (df["AskPrice_5"] == 0) & (df["AskVolume_5"] == 0) \
      & (df["AskPrice_6"] == 0) & (df["AskVolume_6"] == 0) & (df["AskPrice_7"] == 0) & (df["AskVolume_7"] == 0) \
      & (df["AskPrice_8"] == 0) & (df["AskVolume_8"] == 0) & (df["AskPrice_9"] > 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Market/Cancel"
df.loc[a,["Quote"]] = 9
df.loc[a,["Sign"]] = -1
df.loc[a,["Price"]] = clean_data.loc[a-1]["AskPrice_9"].to_list()
df.loc[a,["Volume"]] = clean_data.loc[a-1]["AskVolume_9"].to_list()

# AskPrice_9 < 0
con = (df["AskPrice_0"] == 0) & (df["AskVolume_0"] == 0) & (df["AskPrice_1"] == 0) & (df["AskVolume_1"] == 0) \
      & (df["AskPrice_2"] == 0) & (df["AskVolume_2"] == 0) & (df["AskPrice_3"] == 0) & (df["AskVolume_3"] == 0) \
      & (df["AskPrice_4"] == 0) & (df["AskVolume_4"] == 0) & (df["AskPrice_5"] == 0) & (df["AskVolume_5"] == 0) \
      & (df["AskPrice_6"] == 0) & (df["AskVolume_6"] == 0) & (df["AskPrice_7"] == 0) & (df["AskVolume_7"] == 0) \
      & (df["AskPrice_8"] == 0) & (df["AskVolume_8"] == 0) & (df["AskPrice_9"] < 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Limit"
df.loc[a,["Quote"]] = 9
df.loc[a,["Sign"]] = -1
df.loc[a,["Price"]] = clean_data.loc[a]["AskPrice_9"]
df.loc[a,["Volume"]] = clean_data.loc[a]["AskVolume_9"]


# BidVolume_9 > 0
con = (df["BidPrice_0"] == 0) & (df["BidVolume_0"] == 0) & (df["BidPrice_1"] == 0) & (df["BidVolume_1"] == 0) \
      & (df["BidPrice_2"] == 0) & (df["BidVolume_2"] == 0) & (df["BidPrice_3"] == 0) & (df["BidVolume_3"] == 0) \
      & (df["BidPrice_4"] == 0) & (df["BidVolume_4"] == 0) & (df["BidPrice_5"] == 0) & (df["BidVolume_5"] == 0) \
      & (df["BidPrice_6"] == 0) & (df["BidVolume_6"] == 0) & (df["BidPrice_7"] == 0) & (df["BidVolume_7"] == 0) \
      & (df["BidPrice_8"] == 0) & (df["BidVolume_8"] == 0) & (df["BidPrice_9"] == 0) & (df["BidVolume_9"] > 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Limit"
df.loc[a,["Quote"]] = 9
df.loc[a,["Sign"]] = 1
df.loc[a,["Price"]] = clean_data.loc[a]["BidPrice_9"]
df.loc[a,["Volume"]] = df.loc[a]["BidVolume_9"]

# BidVolume_9 < 0
con = (df["BidPrice_0"] == 0) & (df["BidVolume_0"] == 0) & (df["BidPrice_1"] == 0) & (df["BidVolume_1"] == 0) \
      & (df["BidPrice_2"] == 0) & (df["BidVolume_2"] == 0) & (df["BidPrice_3"] == 0) & (df["BidVolume_3"] == 0) \
      & (df["BidPrice_4"] == 0) & (df["BidVolume_4"] == 0) & (df["BidPrice_5"] == 0) & (df["BidVolume_5"] == 0) \
      & (df["BidPrice_6"] == 0) & (df["BidVolume_6"] == 0) & (df["BidPrice_7"] == 0) & (df["BidVolume_7"] == 0) \
      & (df["BidPrice_8"] == 0) & (df["BidVolume_8"] == 0) & (df["BidPrice_9"] == 0) & (df["BidVolume_9"] < 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Market/Cancel"
df.loc[a,["Quote"]] = 9
df.loc[a,["Sign"]] = 1
df.loc[a,["Price"]] = clean_data.loc[a]["BidPrice_9"]
df.loc[a,["Volume"]] = df.loc[a]["BidVolume_9"] * (-1)

# AskVolume_9 > 0
con = (df["AskPrice_0"] == 0) & (df["AskVolume_0"] == 0) & (df["AskPrice_1"] == 0) & (df["AskVolume_1"] == 0) \
      & (df["AskPrice_2"] == 0) & (df["AskVolume_2"] == 0) & (df["AskPrice_3"] == 0) & (df["AskVolume_3"] == 0) \
      & (df["AskPrice_4"] == 0) & (df["AskVolume_4"] == 0) & (df["AskPrice_5"] == 0) & (df["AskVolume_5"] == 0) \
      & (df["AskPrice_6"] == 0) & (df["AskVolume_6"] == 0) & (df["AskPrice_7"] == 0) & (df["AskVolume_7"] == 0) \
      & (df["AskPrice_8"] == 0) & (df["AskVolume_8"] == 0) & (df["AskPrice_9"] == 0) & (df["AskVolume_9"] > 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Limit"
df.loc[a,["Quote"]] = 9
df.loc[a,["Sign"]] = -1
df.loc[a,["Price"]] = clean_data.loc[a]["AskPrice_9"]
df.loc[a,["Volume"]] = df.loc[a]["AskVolume_9"]

# AskVolume_9 < 0
con = (df["AskPrice_0"] == 0) & (df["AskVolume_0"] == 0) & (df["AskPrice_1"] == 0) & (df["AskVolume_1"] == 0) \
      & (df["AskPrice_2"] == 0) & (df["AskVolume_2"] == 0) & (df["AskPrice_3"] == 0) & (df["AskVolume_3"] == 0) \
      & (df["AskPrice_4"] == 0) & (df["AskVolume_4"] == 0) & (df["AskPrice_5"] == 0) & (df["AskVolume_5"] == 0) \
      & (df["AskPrice_6"] == 0) & (df["AskVolume_6"] == 0) & (df["AskPrice_7"] == 0) & (df["AskVolume_7"] == 0) \
      & (df["AskPrice_8"] == 0) & (df["AskVolume_8"] == 0) & (df["AskPrice_9"] == 0) & (df["AskVolume_9"] < 0)
a = df.loc[con].index.to_numpy()
df.loc[a,["Type"]] = "Market/Cancel"
df.loc[a,["Quote"]] = 9
df.loc[a,["Sign"]] = -1
df.loc[a,["Price"]] = clean_data.loc[a]["AskPrice_9"]
df.loc[a,["Volume"]] = df.loc[a]["AskVolume_9"]* (-1)

In [14]:
df["Quote"].value_counts()

NoBest    36718
0         35246
1         18762
2         13810
3         10841
4         10385
5          9776
6          9575
7          9415
8          8669
9          6817
Name: Quote, dtype: int64

In [15]:
new_df = df.loc[:,["Price", "Volume", "Sign", "Quote", "Type"]]
new_df["DateTime"] = clean_data["Datetime"]
seconds = np.zeros(len(df))
for i, date in enumerate(new_df["DateTime"]):
  seconds[i] = date.second + 60 * date.minute + 3600 * date.hour + \
                date.microsecond * 1e-6 + date.day * 3600 * 24

new_df["Seconds"] = seconds

In [16]:
#@title import trade data
#create a list of all the files in the folder
DIR_1 = "Zero-intelligence-simulations/data/energia/trade_ottobre2021_nuovo/trade_ottobre2021/"
trade_files =os.listdir(DIR_1)
trade_files.sort()
lst_trade = []
for element in trade_files:
    # import data
    df = pd.read_csv(DIR_1 + element)

    df.drop(columns = ["Unnamed: 0"], inplace = True)

    # transform the column Datetime from string to datetime
    df["DateTime"]= pd.to_datetime(df["DateTime"])

    seconds = np.zeros(len(df))
    for i, date in enumerate(df["DateTime"]):
      seconds[i] = date.second + 60 * date.minute + 3600 * date.hour + \
                  date.microsecond * 1e-6 + date.day * 3600 * 24

    df["Seconds"] = seconds

    df = df.loc[df["DateTime"].dt.hour > 6]
    df = df.loc[df["DateTime"].dt.hour < 16]
    
    df = df.loc[df["DateTime"].dt.day != 16]
    lst_trade.append(df)



trade_df = pd.concat(lst_trade)

In [18]:
#@title Match: Tempo Segno Volume Prezzo (1)

df1 = new_df.copy()

lst_index = []
for i in range(len(trade_df)):
  p = trade_df["Price"].iat[i]
  v = trade_df["Volume"].iat[i]
  s = trade_df["Seconds"].iat[i]
  if trade_df["AggressorAction"].iat[i] == "Sell":
    sign = 1
  else:
    sign = -1

  k = 1
  a = []
 #verify if the broker is EEX in this case information on the sign is useless
  if trade_df["AggressorBroker"].iat[i] == "EEX":
    while a == [] and k < 5:
      a = df1[(df1["Volume"] == v) & (df1["Price"] == p) & (df1["Type"] == "Market/Cancel") \
          & (df1["Seconds"] > s - k) & (df1["Seconds"] < s + k)].index.to_list()
      k += 1

  else:
    while a == [] and k < 5:
      a = df1[(df1["Volume"] == v) & (df1["Price"] == p) & (df1["Type"] == "Market/Cancel") \
            & (df1["Seconds"] > s - k) & (df1["Seconds"] < s + k) & (df1["Sign"] == sign)].index.to_list()
      k += 1

  lst_index.append(a)

n1 = 0
for element in lst_index:
  if element == []:
    n1 +=1

df1["Broker"] = ["None" for i in range(len(df1))]
df1["BrokerSign"] = ["None" for i in range(len(df1))]
df1["PriceTrade"] = [0 for i in range(len(df1))]
df1["BrokerVolume"] = [0 for i in range(len(df1))]

for k,element in enumerate(lst_index):
  if element != []:
    # check if the random order was not preavously chosen
    flag = False
    # to avoid infinite loop repeat while at most 10 times
    i = 0
    while flag is False:
      trade = np.random.choice(element)
      if df1["Type"].at[trade] != "Market" or i>10:
        flag = True
      i += 1

    #check if trades was at the best quotes
    df1["Type"].at[trade] = "Market"
    # save broker, price and sign
    df1["Broker"].at[trade] = trade_df["AggressorBroker"].iat[k]
    df1["BrokerSign"].at[trade] =  trade_df["AggressorAction"].iat[k]
    df1["PriceTrade"].at[trade] =  trade_df["Price"].iat[k]
    df1["BrokerVolume"].at[trade] =  trade_df["Volume"].iat[k]

# all remaining market/cancel are cancel order
df1["Type"].replace(["Market/Cancel"], "Cancel", inplace = True)

In [19]:
df1["Type"].value_counts()

Limit     67365
Cancel    60831
0.0       36718
Market     5100
Name: Type, dtype: int64

In [20]:
df1.to_csv("tempo_segno_volume_prezzo.csv")

In [21]:
#@title Match: Tempo Volume Prezzo (2)

df2 = new_df.copy()

lst_index = []
for i in range(len(trade_df)):
  p = trade_df["Price"].iat[i]
  v = trade_df["Volume"].iat[i]
  s = trade_df["Seconds"].iat[i]

  k = 1
  a = []
  #verify if the broker is EEX in this case information on the sign is useless
  while a == [] and k < 5:
    a = df2[(df2["Volume"] == v) & (df2["Price"] == p) & (df2["Type"] == "Market/Cancel") \
          & (df2["Seconds"] > s - k) & (df2["Seconds"] < s + k)].index.to_list()
    k += 1
  lst_index.append(a)

n2 = 0
for element in lst_index:
  if element == []:
    n2 +=1

df2["Broker"] = ["None" for i in range(len(df2))]
df2["BrokerSign"] = ["None" for i in range(len(df2))]
df2["PriceTrade"] = [0 for i in range(len(df2))]
df2["BrokerVolume"] = [0 for i in range(len(df2))]

for k,element in enumerate(lst_index):
  if element != []:
    # check if the random order was not preavously chosen
    flag = False
    # to avoid infinite loop repeat while at most 10 times
    i = 0
    while flag is False:
      trade = np.random.choice(element)
      if df2["Type"].at[trade] != "Market" or i>10:
        flag = True
      i += 1

    #check if trades was at the best quotes
    df2["Type"].at[trade] = "Market"
    # save broker, price and sign
    df2["Broker"].at[trade] = trade_df["AggressorBroker"].iat[k]
    df2["BrokerSign"].at[trade] =  trade_df["AggressorAction"].iat[k]
    df2["PriceTrade"].at[trade] =  trade_df["Price"].iat[k]
    df2["BrokerVolume"].at[trade] =  trade_df["Volume"].iat[k]

# all remaining market/cancel are cancel order
df2["Type"].replace(["Market/Cancel"], "Cancel", inplace = True)

In [22]:
df2.to_csv("tempo_volume_prezzo.csv")

In [23]:
#@title Match: Tempo Prezzo (3)

df3 = new_df.copy()

lst_index = []
for i in range(len(trade_df)):
  p = trade_df["Price"].iat[i]
  s = trade_df["Seconds"].iat[i]

  k = 1
  a = []
  #verify if the broker is EEX in this case information on the sign is useless
  while a == [] and k < 5:
    a = df3[(df3["Price"] == p) & (df3["Type"] == "Market/Cancel") \
        & (df3["Seconds"] > s - k) & (df3["Seconds"] < s + k)].index.to_list()
    k += 1

  lst_index.append(a)

n3 = 0
for element in lst_index:
  if element == []:
    n3 +=1

df3["Broker"] = ["None" for i in range(len(df3))]
df3["BrokerSign"] = ["None" for i in range(len(df3))]
df3["PriceTrade"] = [0 for i in range(len(df3))]
df3["BrokerVolume"] = [0 for i in range(len(df3))]

for k,element in enumerate(lst_index):
  if element != []:
    # check if the random order was not preavously chosen
    flag = False
    # to avoid infinite loop repeat while at most 10 times
    i = 0
    while flag is False:
      trade = np.random.choice(element)
      if df3["Type"].at[trade] != "Market" or i>10:
        flag = True
      i += 1

    #check if trades was at the best quotes
    df3["Type"].at[trade] = "Market"
    # save broker, price and sign
    df3["Broker"].at[trade] = trade_df["AggressorBroker"].iat[k]
    df3["BrokerSign"].at[trade] =  trade_df["AggressorAction"].iat[k]
    df3["PriceTrade"].at[trade] =  trade_df["Price"].iat[k]
    df3["BrokerVolume"].at[trade] =  trade_df["Volume"].iat[k]

# all remaining market/cancel are cancel order
df3["Type"].replace(["Market/Cancel"], "Cancel", inplace = True)

In [24]:
df3.to_csv("tempo_prezzo.csv")

In [25]:
#@title Match: Tempo Volume (4)

df4 = new_df.copy()

lst_index = []
for i in range(len(trade_df)):
  v = trade_df["Volume"].iat[i]
  s = trade_df["Seconds"].iat[i]

  k = 1
  a = []
  #verify if the broker is EEX in this case information on the sign is useless
  while a == [] and k < 5:
    a = df4[ (df4["Volume"] == v) & (df4["Type"] == "Market/Cancel") \
        & (df4["Seconds"] > s - k) & (df4["Seconds"] < s + k)].index.to_list()
    k += 1

  lst_index.append(a)

n4 = 0
for element in lst_index:
  if element == []:
    n4 +=1

df4["Broker"] = ["None" for i in range(len(df4))]
df4["BrokerSign"] = ["None" for i in range(len(df4))]
df4["PriceTrade"] = [0 for i in range(len(df4))]
df4["BrokerVolume"] = [0 for i in range(len(df4))]

for k,element in enumerate(lst_index):
  if element != []:
    # check if the random order was not preavously chosen
    flag = False
    # to avoid infinite loop repeat while at most 10 times
    i = 0
    while flag is False:
      trade = np.random.choice(element)
      if df4["Type"].at[trade] != "Market" or i>10:
        flag = True
      i += 1

    #check if trades was at the best quotes
    df4["Type"].at[trade] = "Market"
    # save broker, price and sign
    df4["Broker"].at[trade] = trade_df["AggressorBroker"].iat[k]
    df4["BrokerSign"].at[trade] =  trade_df["AggressorAction"].iat[k]
    df4["PriceTrade"].at[trade] =  trade_df["Price"].iat[k]
    df4["BrokerVolume"].at[trade] =  trade_df["Volume"].iat[k]

# all remaining market/cancel are cancel order
df4["Type"].replace(["Market/Cancel"], "Cancel", inplace = True)

In [26]:
df4.to_csv("tempo_volume.csv")

In [27]:
#@title Match: Tempo Segno (5)

df5 = new_df.copy()

lst_index = []
for i in range(len(trade_df)):
  v = trade_df["Volume"].iat[i]
  s = trade_df["Seconds"].iat[i]

  k = 1
  a = []
  #verify if the broker is EEX in this case information on the sign is useless
  if trade_df["AggressorBroker"].iat[i] == "EEX":
    while a == [] and k < 5:
      a = df5[(df5["Type"] == "Market/Cancel") \
          & (df5["Seconds"] > s - k) & (df5["Seconds"] < s + k)].index.to_list()
      k += 1

  else:
    while a == [] and k < 5:
      a = df5[(df5["Type"] == "Market/Cancel") & (df5["Seconds"] > s - k) \
               & (df5["Seconds"] < s + k) & (df5["Sign"] == sign)].index.to_list()
      k += 1

  lst_index.append(a)

n5 = 0
for element in lst_index:
  if element == []:
    n5 +=1

df5["Broker"] = ["None" for i in range(len(df5))]
df5["BrokerSign"] = ["None" for i in range(len(df5))]
df5["PriceTrade"] = [0 for i in range(len(df5))]
df5["BrokerVolume"] = [0 for i in range(len(df5))]

for k,element in enumerate(lst_index):
  if element != []:
    # check if the random order was not preavously chosen
    flag = False
    # to avoid infinite loop repeat while at most 10 times
    i = 0
    while flag is False:
      trade = np.random.choice(element)
      if df5["Type"].at[trade] != "Market" or i>10:
        flag = True
      i += 1

    #check if trades was at the best quotes
    df5["Type"].at[trade] = "Market"
    # save broker, price and sign
    df5["Broker"].at[trade] = trade_df["AggressorBroker"].iat[k]
    df5["BrokerSign"].at[trade] =  trade_df["AggressorAction"].iat[k]
    df5["PriceTrade"].at[trade] =  trade_df["Price"].iat[k]
    df5["BrokerVolume"].at[trade] =  trade_df["Volume"].iat[k]

# all remaining market/cancel are cancel order
df5["Type"].replace(["Market/Cancel"], "Cancel", inplace = True)

In [28]:
df5.to_csv("tempo_segno.csv")

In [29]:
#@title Match: Tempo Segno Prezzo (6)

df6 = new_df.copy()

lst_index = []
for i in range(len(trade_df)):
  p = trade_df["Price"].iat[i]
  s = trade_df["Seconds"].iat[i]

  k = 1
  a = []
  #verify if the broker is EEX in this case information on the sign is useless
  if trade_df["AggressorBroker"].iat[i] == "EEX":
    while a == [] and k < 5:
      a = df6[(df6["Type"] == "Market/Cancel") & (df6["Price"] == p)\
          & (df6["Seconds"] > s - k) & (df6["Seconds"] < s + k)].index.to_list()
      k += 1

  else:
    while a == [] and k < 5:
      a = df6[(df6["Type"] == "Market/Cancel") & (df6["Seconds"] > s - k) & (df6["Price"] == p) \
               & (df6["Seconds"] < s + k) & (df6["Sign"] == sign)].index.to_list()
      k += 1

  lst_index.append(a)

n6 = 0
for element in lst_index:
  if element == []:
    n6 +=1

df6["Broker"] = ["None" for i in range(len(df6))]
df6["BrokerSign"] = ["None" for i in range(len(df6))]
df6["PriceTrade"] = [0 for i in range(len(df6))]
df6["BrokerVolume"] = [0 for i in range(len(df6))]

for k,element in enumerate(lst_index):
  if element != []:
    # check if the random order was not preavously chosen
    flag = False
    # to avoid infinite loop repeat while at most 10 times
    i = 0
    while flag is False:
      trade = np.random.choice(element)
      if df6["Type"].at[trade] != "Market" or i>10:
        flag = True
      i += 1

    #check if trades was at the best quotes
    df6["Type"].at[trade] = "Market"
    # save broker, price and sign
    df6["Broker"].at[trade] = trade_df["AggressorBroker"].iat[k]
    df6["BrokerSign"].at[trade] =  trade_df["AggressorAction"].iat[k]
    df6["PriceTrade"].at[trade] =  trade_df["Price"].iat[k]
    df6["BrokerVolume"].at[trade] =  trade_df["Volume"].iat[k]

# all remaining market/cancel are cancel order
df6["Type"].replace(["Market/Cancel"], "Cancel", inplace = True)

In [30]:
df6.to_csv("tempo_segno_prezzo.csv")

In [32]:
#@title Match: Tempo Segno Volume (7)

df7 = new_df.copy()

lst_index = []
for i in range(len(trade_df)):
  v = trade_df["Volume"].iat[i]
  s = trade_df["Seconds"].iat[i]

  k = 1
  a = []
  #verify if the broker is EEX in this case information on the sign is useless
  if trade_df["AggressorBroker"].iat[i] == "EEX":
    while a == [] and k < 5:
      a = df7[(df7["Type"] == "Market/Cancel") & (df7["Volume"] == v)\
          & (df7["Seconds"] > s - k) & (df7["Seconds"] < s + k)].index.to_list()
      k += 1

  else:
    while a == [] and k < 5:
      a = df7[(df7["Type"] == "Market/Cancel") & (df7["Seconds"] > s - k) & (df7["Volume"] == v) \
               & (df7["Seconds"] < s + k) & (df7["Sign"] == sign)].index.to_list()
      k += 1

  lst_index.append(a)

n7 = 0
for element in lst_index:
  if element == []:
    n7 +=1

df7["Broker"] = ["None" for i in range(len(df7))]
df7["BrokerSign"] = ["None" for i in range(len(df7))]
df7["PriceTrade"] = [0 for i in range(len(df7))]
df7["BrokerVolume"] = [0 for i in range(len(df7))]

for k,element in enumerate(lst_index):
  if element != []:
    # check if the random order was not preavously chosen
    flag = False
    # to avoid infinite loop repeat while at most 10 times
    i = 0
    while flag is False:
      trade = np.random.choice(element)
      if df7["Type"].at[trade] != "Market" or i>10:
        flag = True
      i += 1

    #check if trades was at the best quotes
    df7["Type"].at[trade] = "Market"
    # save broker, price and sign
    df7["Broker"].at[trade] = trade_df["AggressorBroker"].iat[k]
    df7["BrokerSign"].at[trade] =  trade_df["AggressorAction"].iat[k]
    df7["PriceTrade"].at[trade] =  trade_df["Price"].iat[k]
    df7["BrokerVolume"].at[trade] =  trade_df["Volume"].iat[k]

# all remaining market/cancel are cancel order
df7["Type"].replace(["Market/Cancel"], "Cancel", inplace = True)

In [34]:
df7.to_csv("tempo_segno_volume.csv")

In [35]:
#@title Match: Tempo (8)

df8 = new_df.copy()

lst_index = []
for i in range(len(trade_df)):
  v = trade_df["Volume"].iat[i]
  s = trade_df["Seconds"].iat[i]

  k = 1
  a = []
  
  while a == [] and k < 5:
    a = df8[(df8["Type"] == "Market/Cancel") & (df8["Seconds"] > s - k)\
            & (df8["Seconds"] < s + k)].index.to_list()
    k += 1

  lst_index.append(a)

n8 = 0
for element in lst_index:
  if element == []:
    n8 +=1

df8["Broker"] = ["None" for i in range(len(df8))]
df8["BrokerSign"] = ["None" for i in range(len(df8))]
df8["PriceTrade"] = [0 for i in range(len(df8))]
df8["BrokerVolume"] = [0 for i in range(len(df8))]

for k,element in enumerate(lst_index):
  if element != []:
    # check if the random order was not preavously chosen
    flag = False
    # to avoid infinite loop repeat while at most 10 times
    i = 0
    while flag is False:
      trade = np.random.choice(element)
      if df8["Type"].at[trade] != "Market" or i>10:
        flag = True
      i += 1

    #check if trades was at the best quotes
    df8["Type"].at[trade] = "Market"
    # save broker, price and sign
    df8["Broker"].at[trade] = trade_df["AggressorBroker"].iat[k]
    df8["BrokerSign"].at[trade] =  trade_df["AggressorAction"].iat[k]
    df8["PriceTrade"].at[trade] =  trade_df["Price"].iat[k]
    df8["BrokerVolume"].at[trade] =  trade_df["Volume"].iat[k]

# all remaining market/cancel are cancel order
df8["Type"].replace(["Market/Cancel"], "Cancel", inplace = True)

In [36]:
df8.to_csv("tempo.csv")

In [37]:
gg = [n1, n2,n3,n4,n5,n6,n7,n8]
gg

[4334, 3683, 3270, 2496, 2707, 4637, 3718, 1536]

In [41]:
new_df.to_csv("order.csv")